## Load Packages

In [ ]:
%pylab inline
import scipy
import h5py
import skimage
import os
from skimage import io,transform,img_as_float
from skimage.io import imread,imsave
from collections import OrderedDict
import decimal
notebook_dir = os.getcwd()
project_dir = os.path.split(notebook_dir)[0]
result_dir = project_dir + '/Results/Images/'
if not os.path.isdir(result_dir):
    os.makedirs(result_dir)
tmp_dir = project_dir + '/Tmp/'
if not os.path.isdir(tmp_dir):
    os.makedirs(tmp_dir)
photo_dir = project_dir + '/Images/ControlPaper/'
art_dir = project_dir + '/Images/ControlPaper/'

## Define Python helper functions - should move to python packge

In [ ]:
def make_torch_input(filename, layers, loss_functions, args):
    f = h5py.File(filename,'w')
    for l,layer in enumerate(layers):
        layer_group = f.create_group(layer)
        for lf,loss_function in enumerate(loss_functions[l]):
            lf_group = layer_group.create_group(loss_function)
            for arg in args[l][lf]:
                dataset = lf_group.create_dataset(arg, data=args[l][lf][arg])
    f.close()
    
def make_torch_init(filename, init):
    f = h5py.File(filename,'w')
    f.create_dataset('init', data=init)
    f.close()

def get_torch_output(filename):
    f = h5py.File(filename,'r')
    data = f['opt_result']
    return data.value
    f.close()
def get_torch_loss(filename):
    f = h5py.File(filename,'r')
    data = f['losses']
    return data.value
    f.close()

def list2css(layers):
    '''
    Takes list of strings and returns comma separated string
    '''
    css = str()
    for l in layers:
        css = css+l+','
    return css[:-1]

def get_activations(images, caffe_model, layers='all', gpu=0):
    '''
    Function to get neural network activations in response to images from torch.
    
    :param images: array of images
    :param caffe_model: file name of the network .caffemodel file
    :param layers: network layers for which the activations should be computed
    :return: network activations in response to images
    '''
    layers = list2css(layers)
    images_file_name = tmp_dir + 'images.hdf5'
    output_file_name = tmp_dir + 'activations.hdf5'
    f = h5py.File(images_file_name, 'w')
    f.create_dataset('images', data=images)
    f.close()
    context = {
    'caffe_model': caffe_model,
    'images': images_file_name,
    'layers': layers,
    'gpu': gpu,
    'backend': 'cudnn',
    'output_file': output_file_name,
    'project_dir': project_dir
    }
    template = ('#!/bin/bash\n' +
                'cd {project_dir} && ' + 
                '/usr/local/torch/install/bin/th ComputeActivations.lua ' + 
                '-caffe_model {caffe_model} ' +
                '-images {images} ' + 
                '-layers {layers} ' + 
                '-gpu {gpu} ' + 
                '-backend {backend} ' +
                '-output_file {output_file}')
    script_name = project_dir + 'get_activations.sh'
    with open(script_name, 'w') as script:
        script.write(template.format(**context))
    os.chmod(script_name, 0o755)
    #execute script 
    !{script_name} >/dev/null
    f = h5py.File(output_file_name,'r')
    act = OrderedDict()
    for key in f.keys():
        act[key] = f[key].value.copy()
    f.close()
    return act

def preprocess(image):
    assert(image.max() <= 1.001)
    imagenet_mean = array([0.40760392,  0.45795686,  0.48501961])
    image_torch = 255 * (image[:,:,::-1] - imagenet_mean).transpose(2,0,1)
    return image_torch

def deprocess(image_torch):
    imagenet_mean = array([0.40760392,  0.45795686,  0.48501961])
    image = (image_torch.transpose(1,2,0)/255. + imagenet_mean)[:,:,::-1]
    image[image>1] = 1
    image[image<0] = 0
    return image

def gram_matrix(activations):
    n_fm = activations.shape[0]
    F = activations.reshape(n_fm,-1)
    G = F.dot(F.T) / F[0,:].size
    return G

import itertools
def flatten(l):
    return list(itertools.chain.from_iterable(l))

def set_model(name, project_dir):
    if name == 'org_pad':
        model = project_dir + '/Models/VGG_ILSVRC_19_layers_conv.caffemodel'
    elif name == 'norm_pad':
        model = project_dir + '/Models/vgg_normalised.caffemodel'
    else:
        assert False, 'unknown model name'
    return model

def match_color(target_img, source_img, mode='sym', eps=1e-5):
    '''
    Matches the colour distribution of the target image to that of the source image
    using a linear transform.
    Images are expected to be of form (w,h,c).
    Modes are chol, pca or sym for different choices of basis.
    '''
    mu_t = target_img.mean(0).mean(0)
    t = target_img - mu_t
    t = t.transpose(2,0,1).reshape(3,-1)
    Ct = t.dot(t.T) / t.shape[1] + eps * eye(t.shape[0])
    mu_s = source_img.mean(0).mean(0)
    s = source_img - mu_s
    s = s.transpose(2,0,1).reshape(3,-1)
    Cs = s.dot(s.T) / s.shape[1] + eps * eye(s.shape[0])
    if mode == 'chol':
        chol_t = np.linalg.cholesky(Ct)
        chol_s = np.linalg.cholesky(Cs)
        ts = chol_s.dot(np.linalg.inv(chol_t)).dot(t)
    if mode == 'pca':
        eva_t, eve_t = np.linalg.eigh(Ct)
        Qt = eve_t.dot(np.sqrt(np.diag(eva_t))).dot(eve_t.T)
        eva_s, eve_s = np.linalg.eigh(Cs)
        Qs = eve_s.dot(np.sqrt(np.diag(eva_s))).dot(eve_s.T)
        ts = Qs.dot(np.linalg.inv(Qt)).dot(t)
    if mode == 'sym':
        eva_t, eve_t = np.linalg.eigh(Ct)
        Qt = eve_t.dot(np.sqrt(np.diag(eva_t))).dot(eve_t.T)
        Qt_Cs_Qt = Qt.dot(Cs).dot(Qt)
        eva_QtCsQt, eve_QtCsQt = np.linalg.eigh(Qt_Cs_Qt)
        QtCsQt = eve_QtCsQt.dot(np.sqrt(np.diag(eva_QtCsQt))).dot(eve_QtCsQt.T)
        ts = np.linalg.inv(Qt).dot(QtCsQt).dot(np.linalg.inv(Qt)).dot(t)
    matched_img = ts.reshape(*target_img.transpose(2,0,1).shape).transpose(1,2,0)
    matched_img += mu_s
    matched_img[matched_img>1] = 1
    matched_img[matched_img<0] = 0
    return matched_img

def lum_transform(image):
    """
    Returns the projection of a colour image onto the luminance channel
    Images are expected to be of form (w,h,c) and float in [0,1].
    """
    img = image.transpose(2,0,1).reshape(3,-1)
    lum = np.array([.299, .587, .114]).dot(img).squeeze()
    img = tile(lum[None,:],(3,1)).reshape((3,image.shape[0],image.shape[1]))
    return img.transpose(1,2,0)

def rgb2luv(image):
    img = image.transpose(2,0,1).reshape(3,-1)
    luv = np.array([[.299, .587, .114],[-.147, -.288, .436],[.615, -.515, -.1]]).dot(img).reshape((3,image.shape[0],image.shape[1]))
    return luv.transpose(1,2,0)
def luv2rgb(image):
    img = image.transpose(2,0,1).reshape(3,-1)
    rgb = np.array([[1, 0, 1.139],[1, -.395, -.580],[1, 2.03, 0]]).dot(img).reshape((3,image.shape[0],image.shape[1]))
    return rgb.transpose(1,2,0)

## org net 

In [ ]:
cp_mode = 'match' #match|lum
img_dirs = OrderedDict()
img_dirs['content'] = photo_dir
img_dirs['style'] = photo_dir
img_names = OrderedDict()
img_names['content'] = 'fig3_content.jpg'
img_names['style'] = 'fig3_style1.jpg'
layers = OrderedDict()
layers['content'] =  ['relu4_2']
layers['style'] = ['relu1_1','relu2_1','relu3_1','relu4_1','relu5_1']
layers_now = layers['style'] + layers['content']
img_size = 512
max_iter = 500
hr_img_size = None
hr_max_iter = 200
gpu = 0
init = 'image'
model_name = 'org_pad'
caffe_model = set_model(model_name, project_dir)
input_file_name = tmp_dir + 'input_cc.hdf5'
init_file_name = tmp_dir + 'init_cc.hdf5'
output_file_name = tmp_dir + 'output_cc.hdf5'
sw = 1e3
cw = 1
weights = OrderedDict()
weights['style'] = [
    [array([sw/64**2])],
    [array([sw/128**2])],
    [array([sw/256**2])],
    [array([sw/512**2])],
    [array([sw/512**2])],
]
weights['content'] = [[array([cw])]]
loss_functions = [['GramMSE']] * len(layers['style']) + [['MSE']]
lf = list2css(map(str,loss_functions));lf = str.replace(lf,'[','');lf = str.replace(lf,']','')
result_image_name = (
'cimg_' + img_names['content'] + 
'_simg_' + img_names['style'] + 
'_cpm_' + cp_mode + 
'_sz_' + str(img_size) + 
'_model_' + model_name + 
'_layers_' + list2css(layers_now) + 
# '_lf_' + lf +
'_sw_' + '%.1E'%decimal.Decimal(sw) + 
'_cw_' + '%.1E'%decimal.Decimal(cw) + 
'_init_' + init +
'.jpg'
)

### norm net 

In [ ]:
cp_mode = 'lum'
img_dirs = OrderedDict()
img_dirs['content'] = photo_dir
img_dirs['style'] = art_dir
img_names = OrderedDict()
img_names['content'] = 'fig3_content.jpg'
img_names['style'] = 'fig3_style1.jpg'
layers = OrderedDict()
layers['content'] =  ['relu4_2']
layers['style'] = ['relu1_1','relu2_1','relu3_1','relu4_1','relu5_1']
layers_now = layers['style'] + layers['content']
img_size = 512
max_iter = 500
hr_img_size = 1024
hr_max_iter = 200
gpu = 0
init = 'image'
model_name = 'norm_pad'
caffe_model = set_model(model_name, project_dir)
input_file_name = tmp_dir + 'input_cc.hdf5'
init_file_name = tmp_dir + 'init_cc.hdf5'
output_file_name = tmp_dir + 'output_cc.hdf5'
sw = 1e9 / len(layers['style'])
cw = 1e6
ptw = 1e5
weights = OrderedDict()
weights['style'] = [[array([sw])]] * len(layers['style'])
weights['content'] = [[array([cw])]]
loss_functions = [['GramMSE']] * len(layers['style']) + [['MSE']]
lf = list2css(map(str,loss_functions));lf = str.replace(lf,'[','');lf = str.replace(lf,']','')
result_image_name = (
'cimg_' + img_names['content'] + 
'_simg_' + img_names['style'] + 
'_cpm_' + cp_mode + 
'_sz_' + str(img_size) + 
'_model_' + model_name + 
'_layers_' + list2css(layers_now) + 
# '_lf_' + lf +
'_sw_' + '%.1E'%decimal.Decimal(sw) + 
'_cw_' + '%.1E'%decimal.Decimal(cw) + 
'_init_' + init +
'.jpg'
)

In [ ]:
if os.path.isfile(result_dir + result_image_name) == False: #check if file exists
    #get images
    conditions = img_names.keys()
    imgs = OrderedDict()
    imgs_torch = OrderedDict()
    act = OrderedDict()
    for cond in conditions:
        imgs[cond] = img_as_float(imread(img_dirs[cond] + img_names[cond]))
        if imgs[cond].ndim == 2:
            imgs[cond] = tile(imgs[cond][:,:,None],(1,1,3))
        elif imgs[cond].shape[2] == 4:
            imgs[cond] = imgs[cond][:,:,:3]
        try:
            imgs[cond] = transform.pyramid_reduce(imgs[cond], sqrt(float(imgs[cond][:,:,0].size) / img_size**2))
        except:
            print('no downsampling: ' + img_names[cond])
        imshow(imgs[cond]);show()
    #color preservation
    if cp_mode == 'lum':
        org_content = imgs['content'].copy()
        for cond in conditions:
            imgs[cond] = lum_transform(imgs[cond])
        imgs['style'] -= imgs['style'].mean(0).mean(0)
        imgs['style'] += imgs['content'].mean(0).mean(0)
        for cond in conditions:
            imgs[cond][imgs[cond]<0] = 0
            imgs[cond][imgs[cond]>1] = 1
    elif cp_mode =='match':
        imgs['style'] = match_color(imgs['style'], imgs['content'], mode='pca')
    elif cp_mode == 'match_style':
        imgs['content'] = match_color(imgs['content'], imgs['style'], mode='pca')
    else:
        raise NameError('Unknown colour preservation mode')
    for cond in conditions:
        imgs_torch[cond] = preprocess(imgs[cond])
        act[cond] = get_activations(imgs_torch[cond],
                                    caffe_model,
                                    layers=layers['style'],
                                    gpu=gpu
                                   )

    if init == 'image':
        make_torch_init(init_file_name, imgs_torch['content'])
    elif init == 'random':
        make_torch_init(init_file_name, randn(*imgs_torch['content'].shape))
    else: 
        raise NameError('Unknown init')

    args = OrderedDict()
    args['style'] = [
                        [
                            {'targets': gram_matrix(act['style'][layer])[None,:],
                             'weights': weights['style'][l][0]} 
                        ] 
                    for l,layer in enumerate(layers['style'])]
    act['content'] = get_activations(imgs_torch['content'], caffe_model, layers=layers['content'], gpu=gpu)
    args['content'] = [[{'targets': act['content'][layers['content'][0]][None,:],'weights': weights['content'][0][0]}],]                
    args_now = args['style'] + args['content']
    make_torch_input(input_file_name, layers_now, loss_functions, args_now)
    context = {
        'caffe_model': caffe_model,
        'input_file': input_file_name,
        'init_file': init_file_name,
        'gpu': gpu,
        'max_iter': max_iter,
        'backend': 'cudnn',
        'print_iter': 50,
        'save_iter': 0,
        'layer_order': list2css(layers_now),
        'output_file': output_file_name,
        'project_dir': project_dir
    }
    template = (
                '#!/bin/bash\n' +
                'cd {project_dir} && ' + 
                'time /usr/local/torch/install/bin/th ImageSynthesis.lua ' + 
                '-caffe_model {caffe_model} ' +
                '-input_file {input_file} ' + 
                '-init_file {init_file} ' + 
                '-gpu {gpu} ' + 
                '-max_iter {max_iter} ' +
                '-print_iter {print_iter} ' +
                '-save_iter {save_iter} ' +
                '-backend {backend} ' + 
                '-layer_order {layer_order} ' +
                '-output_file {output_file}'
               )

    script_name = project_dir + 'run_synthesis.sh'
    with open(script_name, 'w') as script:
        script.write(template.format(**context))
    os.chmod(script_name, 0o755)
    #execute script 
    !{script_name}
    output = deprocess(get_torch_output(output_file_name))
    if cp_mode == 'lum':
        org_content = rgb2luv(org_content)
        org_content[:,:,0] = output.mean(2)
        output = luv2rgb(org_content)
        output[output<0] = 0
        output[output>1]=1
    imshow(output);gcf().set_size_inches(8,14);show()
    imsave(result_dir + result_image_name, output)

#Make Highres
if hr_img_size: 
    lr_output = img_as_float(imread(result_dir + result_image_name))
    result_image_name = (
    'cimg_' + img_names['content'] + 
    '_simg_' + img_names['style'] + 
    '_cpm_' + cp_mode + 
    '_sz_' + str(img_size) + 
    '_hrsz_' + str(hr_img_size) + 
    '_model_' + model_name + 
    '_layers_' + list2css(layers_now) + 
#     '_lf_' + lf +
    '_sw_' + '%.1E'%decimal.Decimal(sw) + 
    '_cw_' + '%.1E'%decimal.Decimal(cw) +  
    '_init_' + init +
    '.jpg'
    )
    if os.path.isfile(result_dir + result_image_name) == False: #check if file exists
        #get images
        conditions = img_names.keys()
        imgs = OrderedDict()
        imgs_torch = OrderedDict()
        act = OrderedDict()
        for cond in conditions:
            imgs[cond] = img_as_float(imread(img_dirs[cond] + img_names[cond]))
            if imgs[cond].ndim == 2:
                imgs[cond] = tile(imgs[cond][:,:,None],(1,1,3))
            elif imgs[cond].shape[2] == 4:
                imgs[cond] = imgs[cond][:,:,:3]
            try:
                imgs[cond] = transform.pyramid_reduce(imgs[cond], sqrt(float(imgs[cond][:,:,0].size) / hr_img_size**2))
            except:
                print('no downsampling: ' + img_names[cond])
            imshow(imgs[cond]);show()
        #color preservation
        if cp_mode == 'lum':
            org_content = imgs['content'].copy()
            for cond in conditions:
                imgs[cond] = lum_transform(imgs[cond])
            imgs['style'] -= imgs['style'].mean(0).mean(0)
            imgs['style'] += imgs['content'].mean(0).mean(0)
            for cond in conditions:
                imgs[cond][imgs[cond]<0] =0
                imgs[cond][imgs[cond]>1] =1
        elif cp_mode =='match':
            imgs['style'] = match_color(imgs['style'], imgs['content'], mode='pca')
        elif cp_mode == 'match_style':
            imgs['content'] = match_color(imgs['content'], imgs['style'], mode='pca')
        else:
            raise NameError('Unknown colour preservation mode')
        for cond in conditions:
            imgs_torch[cond] = preprocess(imgs[cond])
            act[cond] = get_activations(imgs_torch[cond],
                                        caffe_model,
                                        layers=layers['style'],
                                        gpu=gpu
                                       )

        hr_init = img_as_float(scipy.misc.imresize(lr_output, imgs['content'].shape))
        if cp_mode == 'lum':
            hr_init = lum_transform(hr_init)
        hr_init = preprocess(hr_init)
        make_torch_init(init_file_name, hr_init)

        args = OrderedDict()
        args['style'] = [
                            [
                                {'targets': gram_matrix(act['style'][layer])[None,:],
                                 'weights': weights['style'][l][0]} 
                            ] 
                        for l,layer in enumerate(layers['style'])]
        act['content'] = get_activations(imgs_torch['content'], caffe_model, layers=layers['content'], gpu=gpu)
        args['content'] = [[{'targets': act['content'][layers['content'][0]][None,:],'weights': weights['content'][0][0]}],]                
        layers_now = layers['style'] + layers['content']
        args_now = args['style'] + args['content']

        make_torch_input(input_file_name, layers_now, loss_functions, args_now)
        context = {
            'caffe_model': caffe_model,
            'input_file': input_file_name,
            'init_file': init_file_name,
            'gpu': gpu,
            'max_iter': hr_max_iter,
            'backend': 'cudnn',
            'print_iter': 50,
            'save_iter': 0,
            'layer_order': list2css(layers_now),
            'output_file': output_file_name,
            'project_dir': project_dir
        }
        template = (
                    '#!/bin/bash\n' +
                    'cd {project_dir} && ' + 
                    'time /usr/local/torch/install/bin/th ImageSynthesis.lua ' + 
                    '-caffe_model {caffe_model} ' +
                    '-input_file {input_file} ' + 
                    '-init_file {init_file} ' + 
                    '-gpu {gpu} ' + 
                    '-max_iter {max_iter} ' +
                    '-print_iter {print_iter} ' +
                    '-save_iter {save_iter} ' +
                    '-backend {backend} ' + 
                    '-layer_order {layer_order} ' +
                    '-output_file {output_file}'
                   )

        script_name = project_dir + 'run_synthesis.sh'
        with open(script_name, 'w') as script:
            script.write(template.format(**context))
        os.chmod(script_name, 0o755)
        #execute script 
        !{script_name}
        output = deprocess(get_torch_output(output_file_name))
        if cp_mode == 'lum':
            org_content = rgb2luv(org_content)
            org_content[:,:,0] = output.mean(2)
            output = luv2rgb(org_content)
            output[output<0] = 0
            output[output>1]=1
        imshow(output);gcf().set_size_inches(8,14);show()
        imsave(result_dir + result_image_name, output)